In [1]:
import pandas as pd
import datetime


In [3]:
raw_data = pd.read_csv('../data/raw/waqi-covid19-airqualitydata-2025.csv', skiprows=4)

In [4]:
raw_data.head()

,Date,Country,City,Specie,count,min,max,median,variance
0,2024-07-27,CY,Nicosia,co,48,1.3,4.5,3.0,11.50
1,2025-02-10,CY,Nicosia,co,48,2.2,18.8,4.1,144.33
2,2025-05-07,CY,Nicosia,co,48,0.4,6.5,1.8,14.62
3,2025-05-20,CY,Nicosia,co,48,0.8,3.3,2.7,5.66
4,2025-05-23,CY,Nicosia,co,48,0.6,4.7,1.9,7.71


In [5]:
raw_data.columns

Index(['Date', 'Country', 'City', 'Specie', 'count', 'min', 'max', 'median',
       'variance'],
      dtype='object')

In [8]:
def process_city_data(df):
    # Convert 'Date' column to datetime
    df.loc[:, 'Date'] = pd.to_datetime(df['Date']).dt.date
    
    # Get unique cities
    cities = df[df['Country'] == 'NP']['City'].unique()
    
    # Dictionary to store the processed dataframes
    city_dataframes = {}
    
    for city in cities:
        city_df = df[df['City'] == city]
        
        # Pivot the dataframe to create new columns for each specie
        city_pivot = city_df.pivot_table(index=['Date', 'Country', 'City'], columns='Specie', values='median', aggfunc='first')
        
        # Flatten the multi-level columns
        city_pivot.columns = ['_'.join(col).strip() for col in city_pivot.columns.values]
        
        # Reset the index to make 'Date', 'Country', and 'City' columns again
        city_pivot.reset_index(inplace=True)
        
        # Store the processed dataframe in the dictionary
        city_dataframes[city] = city_pivot
    
    return city_dataframes

# Process the data for all cities in merged_df
city_dataframes = process_city_data(raw_data)

# Example: Access the processed dataframe for Kathmandu
ktm_city_pivot = city_dataframes['Kathmandu']
ktm_city_pivot.head()

,Date,Country,City,d_e_w,h_u_m_i_d_i_t_y,p_m_1,p_m_1_0,p_m_2_5,p_r_e_s_s_u_r_e,t_e_m_p_e_r_a_t_u_r_e,w_i_n_d_-_g_u_s_t,w_i_n_d_-_s_p_e_e_d
0,2024-01-01,NP,Kathmandu,7.5,87.0,113.0,61.0,138.0,1020.0,9.5,NaN,1.2
1,2024-01-02,NP,Kathmandu,6.5,81.5,95.0,63.0,116.0,1020.0,10.0,NaN,1.2
2,2024-01-03,NP,Kathmandu,6.0,81.5,109.0,59.0,131.0,1020.0,9.5,NaN,1.0
3,2024-01-04,NP,Kathmandu,5.0,78.0,103.0,66.0,140.0,1020.0,8.8,NaN,1.0
4,2024-01-05,NP,Kathmandu,4.0,76.0,131.0,62.0,140.0,1020.0,9.5,NaN,1.2


In [9]:
ktm_city_pivot.sort_values('Date', inplace=True)




In [10]:
# Convert the 'Date' column to datetime
ktm_city_pivot['Date'] = pd.to_datetime(ktm_city_pivot['Date'])

# Create a date range from the minimum to the maximum date in the dataframe
full_date_range = pd.date_range(start=ktm_city_pivot['Date'].min(), end=ktm_city_pivot['Date'].max())

# Find the missing dates by comparing the full date range with the dates in the dataframe
missing_dates = full_date_range.difference(ktm_city_pivot['Date'])

missing_dates

DatetimeIndex([], dtype='datetime64[ns]', freq='D')

In [11]:
pokhara_data = city_dataframes['Pokhara']

# Create a date range from the minimum to the maximum date in the dataframe
full_date_range = pd.date_range(start=pokhara_data['Date'].min(), end=pokhara_data['Date'].max())

# Find the missing dates by comparing the full date range with the dates in the dataframe
missing_dates = full_date_range.difference(pokhara_data['Date'])

pokhara_data.isna().sum()

Date                       0
Country                    0
City                       0
d_e_w                    130
h_u_m_i_d_i_t_y            5
p_m_1                    130
p_m_1_0                  124
p_m_2_5                  124
p_r_e_s_s_u_r_e          130
t_e_m_p_e_r_a_t_u_r_e      5
w_i_n_d_-_s_p_e_e_d        6
dtype: int64

In [12]:
Biratnagar_data = city_dataframes['Biratnagar']

# Create a date range from the minimum to the maximum date in the dataframe
full_date_range = pd.date_range(start=Biratnagar_data['Date'].min(), end=Biratnagar_data['Date'].max())

# Find the missing dates by comparing the full date range with the dates in the dataframe
missing_dates = full_date_range.difference(Biratnagar_data['Date'])

Biratnagar_data.shape

missing_dates

DatetimeIndex(['2024-03-08', '2024-05-27'], dtype='datetime64[ns]', freq=None)

### Main code to convert and create the yearly dataset


In [15]:


# Filter data for Nepal
nepal_df = raw_data[raw_data['Country'] == 'NP']

# Get unique cities in Nepal
cities = nepal_df['City'].unique()

# Define the date range for the entire year 
date_range = pd.date_range(start=raw_data['Date'].min(), end=raw_data['Date'].max(), freq='D')

# Function to process and save city data
def process_and_save_city_data(city_df, city_name):
    # Pivot the dataframe to create columns for each specie
    city_pivot = city_df.pivot_table(index='Date', columns='Specie', values='median', aggfunc='first')
    
    # Reindex to include all dates in  and fill missing values
    city_pivot = city_pivot.reindex(date_range)
    
    # Reset index to make 'Date' a column
    city_pivot.reset_index(inplace=True)
    city_pivot.rename(columns={'index': 'Date'}, inplace=True)
    
    # Save the processed data to a CSV file
    city_pivot.to_csv(f'../data/processed/{city_name}_2025.csv', index=False)
    
    return pd.DataFrame(city_pivot)

# Process and save data for each city
for city in cities:
    city_df = nepal_df[nepal_df['City'] == city]
    process_and_save_city_data(city_df, city)


In [20]:
df = pd.read_csv('../data/processed/Kathmandu_2025.csv')

df.drop(columns=['wind-speed','wind-gust'], inplace=True, axis=1)

df.to_csv('../data/processed/Kathmandu_2025.csv', index=False)

In [21]:
df = pd.read_csv('../data/processed/Kathmandu_2025.csv')
df.columns

Index(['Date', 'dew', 'humidity', 'pm1', 'pm10', 'pm25', 'pressure',
       'temperature'],
      dtype='object')